# Machine Learning Recipes #8
## Let’s Write a Decision Tree Classifier from Scratch

```python
#----------------------------------------------------------------------
#### Machine Learning Recipes with Josh Gordon (Google Developers) ####

# Class     Let’s Write a Decision Tree Classifier from Scratch - Machine Learning Recipes #8
# Link      https://youtu.be/LDRbO9a6XPU
# @Code     Josh Gordon		
# @Comments Josh Gordon (original) and Karl Sousa
#----------------------------------------------------------------------
```

In [1]:
# For Python 2 / 3 compatability
from __future__ import print_function

In [3]:
# Toy dataset.
# Format: each row is an example.
# The last column is the label.
# The first two columns are features.
# Feel free to play with it by adding more features & examples.
# Interesting note: I've written this so the 2nd and 5th examples
# have the same features, but different labels - so we can see how the
# tree handles this case.
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [4]:
# Column labels.
# These are used only to print the tree.
header = ["color", "diameter", "label"]

In [7]:
"""Find the unique values for a column in a dataset."""
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [24]:
"""Demo for the function unique_vals"""
# Printing the different types of features for a column of the training data
print(unique_vals(training_data, 0), unique_vals(training_data, 1))

set(['Green', 'Red', 'Yellow']) set([1, 3])


In [12]:
"""Counts the number of each type of example in a dataset."""

def class_counts(rows):
    counts = {}  # a dictionary of label -> count.
    for row in rows:  #gets each example of the dataset
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [15]:
"""Demo for the class counting"""
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [16]:
"""Says if a given variable is numeric"""
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [20]:
"""Demo for the function is_numeric"""
print(is_numeric(7), is_numeric("Red"))

True False


In [26]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [35]:
"""Demo for a numeric question"""
q1 = Question(1, 3)
q1

Is diameter >= 3?

In [36]:
"""Demo for a categorical attribute"""
q2 = Question(0, 'Green')
q2

Is color == Green?

In [48]:
"""Demo for the match method"""
# Let's pick an example from the training set...
example = training_data[0]

print("Example = %s" % example, "%s %s" % (q1, q1.match(example)), "%s %s" % (q2, q2.match(example)), sep = '\n')

Example = ['Green', 3, 'Apple']
Is diameter >= 3? True
Is color == Green? True


In [50]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [53]:
"""Demo for function partition"""
# Let's partition the training data based on whether rows are Red.
true_rows, false_rows = partition(training_data, Question(0, 'Red'))

# The "true rows" will contain all the samples with the feature 'red',
# while the false rows, will contain all the other ones.
print("True rows: %s" % true_rows, "False rows: %s" % false_rows, sep = '\n')

True rows: [['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
False rows: [['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


In [80]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows) # class_counts returns the labels and the their number of elements
    impurity = 1
    for label in counts:
        prob_of_lbl = counts[label] / float(len(rows))  # counts[label] gets the # of elements of a label
        impurity -= prob_of_lbl**2
    return impurity

In [81]:
"""Demo for gini function with no mixing"""

no_mixing = [['Apple'],
              ['Apple']]
# this will return 0
gini(no_mixing)

0.0

In [82]:
"""Demo for the gini function with mixing"""

# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = [['Apple'],
               ['Orange']]
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
gini(some_mixing)

0.5

In [83]:
"""Demo for gini function with mixing"""
# Now, we'll look at a dataset with many different labels
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)

0.7999999999999998

In [84]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [85]:
"""Demo for gini function with mixing"""

# Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999

In [87]:
"""Demo for information gain"""

# How much information do we gain by partioning on 'Green' or 'Red'?
true_rows_green, false_rows_green = partition(training_data, Question(0, 'Green'))
true_rows_red, false_rows_red = partition(training_data, Question(0, 'Red'))

info_gain_green = info_gain(true_rows_green, false_rows_green, current_uncertainty)
info_gain_red = info_gain(true_rows_red, false_rows_red, current_uncertainty)

print("Info gain for 'Red': %s\nInfo gain for 'Green': %s" % (info_gain_green, info_gain_red))

Info gain for 'Red': 0.14
Info gain for 'Green': 0.373333333333


In [89]:
# It looks like we learned more using 'Red' (0.37), than 'Green' (0.14).
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.

# Here, the true_rows contain only 'Grapes'.
# And the false rows contain two types of fruit. Not too bad.
print("'True' splits for red: %s\n'False' splits for red: %s" % (true_rows_red, false_rows_red))

'True' splits for red: [['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
'False' splits for red: [['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


In [90]:
# On the other hand, partitioning by Green doesn't help so much.

# We've isolated one apple in the true rows.
# But, the false-rows are badly mixed up.

print("'True' splits for green: %s\n'False' splits for green: %s" % (true_rows_green, false_rows_green))

'True' splits for green: [['Green', 3, 'Apple']]
'False' splits for green: [['Yellow', 3, 'Apple'], ['Red', 1, 'Grape'], ['Red', 1, 'Grape'], ['Yellow', 3, 'Lemon']]


In [91]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [92]:
"""Demo for the function finding the best question"""

# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is diameter >= 3?

In [93]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [94]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [96]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [97]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [98]:
my_tree = build_tree(training_data)

In [99]:
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Lemon': 1, 'Apple': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [100]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [101]:
"""Demo for classification"""

# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(training_data[0], my_tree)

{'Apple': 1}

In [102]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [104]:
"""Demo for printing the predictions at a leaf """

print_leaf(classify(training_data[0], my_tree))

{'Apple': '100%'}

In [105]:
"""Demo for printing the predictions at a leaf """

# On the second example, the confidence is lower
print_leaf(classify(training_data[1], my_tree))

{'Apple': '50%', 'Lemon': '50%'}

In [106]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [107]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Lemon': '50%', 'Apple': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Lemon': '50%', 'Apple': '50%'}
